# Preprocessing Text Data
The goal of this exploration is to survey and compare various ways of preprocessing text data, mainly with an eye on vector space embeddings.

Let's start by using a cookiecutter approach. There has been some good thought put into this framework, so let's give it a spin and see how well it performs

## Directory Structure
Created via `cookiecutter https://github.com/drivendata/cookiecutter-data-science`

```
├── LICENSE
├── Makefile           <- Makefile with commands like `make data` or `make train`
├── README.md          <- The top-level README for developers using this project.
├── data
│   ├── external       <- Data from third party sources.
│   ├── interim        <- Intermediate data that has been transformed.
│   ├── processed      <- The final, canonical data sets for modeling.
│   └── raw            <- The original, immutable data dump.
│
├── docs               <- A default Sphinx project; see sphinx-doc.org for details
│
├── models             <- Trained and serialized models, model predictions, or model summaries
│
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering),
│                         the creator's initials, and a short `-` delimited description, e.g.
│                         `1.0-jqp-initial-data-exploration`.
│
├── references         <- Data dictionaries, manuals, and all other explanatory materials.
│
├── reports            <- Generated analysis as HTML, PDF, LaTeX, etc.
│   └── figures        <- Generated graphics and figures to be used in reporting
│
├── requirements.txt   <- The requirements file for reproducing the analysis environment, e.g.
│                         generated with `pip freeze > requirements.txt`
│
├── src                <- Source code for use in this project.
│   ├── __init__.py    <- Makes src a Python module
│   │
│   ├── data           <- Scripts to download or generate data
│   │   └── make_dataset.py
│   │
│   ├── features       <- Scripts to turn raw data into features for modeling
│   │   └── build_features.py
│   │
│   ├── models         <- Scripts to train models and then use trained models to make
│   │   │                 predictions
│   │   ├── predict_model.py
│   │   └── train_model.py
│   │
│   └── visualization  <- Scripts to create exploratory and results oriented visualizations
│       └── visualize.py
│
└── tox.ini            <- tox file with settings for running tox; see tox.testrun.org
```

## Pull in the data
We want to sync up our data. There is glue in the default makefile for syncing with s3. 

In [ ]:
!make -f ../Makefile

Okay, this is a beautiful trick - self-documenting Makefiles. I'm going to steal this and put it in all my makefiles from now on: [Self Documenting Makefile Code](https://github.com/drivendata/cookiecutter-data-science/blob/f891ddeeb43a59c9542375efdb2bb4dcc6ebf826/%7B%7B%20cookiecutter.repo_name%20%7D%7D/Makefile#L107-L144)

Next, I added some glue to sync the files somewhere on the filesystem

```
--- a/Makefile
+++ b/Makefile
@@ -10,12 +10,24 @@ PROFILE = default
 PROJECT_NAME = text_preproc
 PYTHON_INTERPRETER = python3
 
+DATASET=yelp-dataset-11
+
+RAW_DATA_PATH=$(DATA_REPO)/raw/$(DATASET)
+EXTERNAL_DATA_PATH=$(DATA_REPO)/external/$(DATASET)
+INTERIM_DATA_PATH=$(DATA_REPO)/interim/$(DATASET)
+PROCESSED_DATA_PATH=$(DATA_REPO)/processed/$(DATASET)
+
 ifeq (,$(shell which conda))
 HAS_CONDA=False
 else
 HAS_CONDA=True
 endif
 
+ifndef DATA_REPO
+$(error DATA_REPO environment var needs to be set)
+endif
+
+
 #################################################################################
 # COMMANDS                                                                      #
 #################################################################################
@@ -46,6 +58,17 @@ else
 	aws s3 sync data/ s3://$(BUCKET)/data/ --profile $(PROFILE)
 endif
 
+## Sync with raw data mountpoint
+pull_data:
+	find $(EXTERNAL_DATA_PATH)/ -name "*json" -exec ln -sf {} data/external/ \;
+	rsync -az $(INTERIM_DATA_PATH)/ data/interim/
+	rsync -az $(PROCESSED_DATA_PATH)/ data/processed/
+
+## Sync with raw data mountpoint
+push_data:
+	rsync -az --delete data/interim/ $(INTERIM_DATA_PATH)/
+	rsync -az --delete data/processed/ $(PROCESSED_DATA_PATH)/
+
 ## Download Data from S3
 sync_data_from_s3:
 ifeq (default,$(PROFILE))
```